In [1]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
from grid import Grid
my_aoc = aoc.AdventOfCode(2018,13)

In [2]:
input_text=r"""/->-\
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/"""
input_lines = input_text.splitlines()

In [7]:
def parse_data(lines):
    data_map = []
    for line in lines:
        data_map.append(list(line))
    new_grid = Grid(data_map, default_value=' ', overrides={})
    return new_grid

class Cart:
    symbols = {
        'n': '^',
        'e': '>',
        's': 'v',
        'w': '<'
    }

    def __init__(self, pos, direction, grid):
        self.pos = pos
        self.direction = direction
        self.grid = grid
        self.grid.overrides[self.pos] = self.symbols[self.direction]
        self.turns = 0
    
    def __lt__(self, other):
        if self.pos[1] < other.pos[1]:
            return True
        if self.pos[1] > other.pos[1]:
            return False
        if self.pos[0] < other.pos[0]:
            return True
        if self.pos[0] > other.pos[0]:
            return False
        return False
    
    def move(self):
        self.grid.pos = self.pos
        neighbors = self.grid.get_neighbors(directions=[self.direction])
        next_pos = neighbors[self.direction]
        self.grid.overrides.pop(self.pos)
        self.pos = next_pos
        curves = {
            'n': {'/': 'e', '\\': 'w'},
            'e': {'/': 'n', '\\': 's'},
            's': {'/': 'w', '\\': 'e'},
            'w': {'/': 's', '\\': 'n'}
        }
        if self.grid.map[next_pos] in '/\\':
            self.direction = curves[self.direction][self.grid.map[next_pos]]
        # Each time a cart has the option to turn (by arriving at any intersection),
        # it turns left the first time, goes straight the second time, turns right the third time,
        # and then repeats those directions starting again with left the fourth time, straight the fifth time,
        # and so on. This process is independent of the particular intersection at which the cart has arrived
        # - that is, the cart has no per-intersection memory.
        directions = 'rlf'
        turns = {
            'n' : {'f': 'n', 'l': 'w', 'r': 'e'},
            'e' : {'f': 'e', 'l': 'n', 'r': 's'},
            's' : {'f': 's', 'l': 'e', 'r': 'w'},
            'w' : {'f': 'w', 'l': 's', 'r': 'n'}
        }
        if self.grid.map[next_pos] in '+':
            self.turns += 1
            turn_direction = directions[self.turns % 3]
            self.direction = turns[self.direction][turn_direction]

        if self.pos in self.grid.overrides:
            # collision
            self.grid.overrides[self.pos] = 'X'
            return False
        else:
            self.grid.overrides[self.pos] =  self.symbols[self.direction]
        return True
        
            

        

        
        
    


In [12]:
grid = parse_data(input_lines)
carts = []
for point in grid.map.keys():
    if grid.map[point] == '^':
        grid.map[point] = '|'
        carts.append(Cart(point, 'n', grid))
    if grid.map[point] == 'v':
        grid.map[point] = '|'
        carts.append(Cart(point, 's', grid))
    if grid.map[point] == '>':
        grid.map[point] = '-'
        carts.append(Cart(point, 'e', grid))
    if grid.map[point] == '<':
        grid.map[point] = '-'
        carts.append(Cart(point, 'w', grid))

collision = False
collision_point = None
#print(grid)
while not collision:
    carts.sort()
    for cart in carts:
        collision = not cart.move()
        if collision:
            collision_point = cart.pos
            break
    #print(grid)
    #print()
print(','.join(str(coord) for coord in collision_point))

    
    
    
        
    

7,3


In [5]:
print(grid)

/---\        
|   |  /----X
| /-+--+-\  |
| | |  | |  |
\-+-/  \-+--/
  \------/   


In [6]:
turns = 0
directions = ['f','l','r']
for _ in range(10):
    turns += 1
    print(turns, directions[turns % 3])


1 l
2 r
3 f
4 l
5 r
6 f
7 l
8 r
9 f
10 l


In [6]:
from collections import deque

q = deque([1,2,3,4,5])
q.rotate(-1)
print(q)
q[0] = 7
print(q)
q.rotate(-1)
print(q)

deque([2, 3, 4, 5, 1])
deque([7, 3, 4, 5, 1])
deque([3, 4, 5, 1, 7])


In [4]:
import itertools

for a, b in itertools.combinations(q, 2):
    print(a,b)


2 3
2 4
2 5
2 1
3 4
3 5
3 1
4 5
4 1
5 1


In [13]:
import timeit
import random

class Cart:
    def __init__(self, pos, direction='n'):
        """
        Init new cart
        """
        self.pos = pos
        self.direction = direction
        self.turns = 0
        self.removed = False
        self.moves = 0

# Generate a dataset of 20 random (x, y) tuples with associated values
data_set = []
for i in range(17):
    x = (random.randint(0, 200))
    y = (random.randint(0, 200))
    data_set.append((x,y,Cart(x,y)))

# Case A: List of tuples
list_of_tuples = data_set.copy()

# Case B: Dictionary with (x, y) as keys and associated values
dict_of_points = {(x, y): val for x, y, val in data_set}

# Sorting function for the list of tuples
def sort_list_of_tuples():
    return sorted(list_of_tuples, key=lambda item: (item[1], item[0]))

# Sorting function for the dictionary
def sort_dict_of_points():
    return sorted(dict_of_points.keys(), key=lambda item: (item[1], item[0]))

# Set up the timeit testing
list_time = timeit.timeit('sort_list_of_tuples()', globals=globals(), number=10000)
dict_time = timeit.timeit('sort_dict_of_points()', globals=globals(), number=10000)

# Print the results
print(f"Time to sort list of tuples: {list_time:.6f} seconds")
print(f"Time to sort dict of points: {dict_time:.6f} seconds")


Time to sort list of tuples: 0.019182 seconds
Time to sort dict of points: 0.019078 seconds
